# Regularization Techniques for Machine Learning

In this notebook, we'll explore regularization techniques to combat overfitting in machine learning models. Regularization adds constraints to model parameters, effectively limiting model complexity while maintaining predictive power.

We'll cover:
1. The problem of overfitting and the need for regularization
2. L2 regularization (Ridge) implementation and effects
3. Parameter tuning for optimal regularization
4. Visualizing how regularization affects model parameters and predictions
5. Comparing regularized vs unregularized models

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge

# Add the parent directory to sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import our implementations
from models.polynomial_regression import PolynomialRegression
from utils.preprocessing import StandardScaler
from utils.plotting import plot_regularization_path

# Set random seed for reproducibility
np.random.seed(42)

## 1. Understanding Overfitting and the Need for Regularization

When a model learns the training data too well, it captures noise rather than underlying patterns. This leads to poor performance on new, unseen data - a problem known as *overfitting*.

Let's generate a dataset to demonstrate this issue:

In [ ]:
def generate_sinusoidal_data(n_samples=50, noise_level=1.0):
    """Generate sinusoidal data with noise"""
    # Generate x values between -3 and 3
    x = np.linspace(-3, 3, n_samples)
    
    # Generate true y values: a sine function
    y_true = np.sin(x) + 0.3*x
    
    # Add noise to create the observed y values
    y = y_true + np.random.normal(0, noise_level, size=n_samples)
    
    return x, y, y_true

# Generate data
x_data, y_data, y_true = generate_sinusoidal_data(n_samples=30, noise_level=0.4)

# Split data
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=42)

# Reshape for our models (expects 2D input)
X_train_2d = X_train.reshape(-1, 1)
X_test_2d = X_test.reshape(-1, 1)

# Plot the data
plt.figure(figsize=(10, 6))
plt.plot(x_data, y_true, 'g-', label='True function', linewidth=2)
plt.scatter(X_train, y_train, color='blue', s=50, alpha=0.7, label='Training data')
plt.scatter(X_test, y_test, color='red', s=50, alpha=0.7, label='Test data')
plt.title('Sinusoidal Dataset with Noise')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()

Now let's demonstrate overfitting by training polynomial regression models with different degrees:

In [ ]:
# Train polynomial models with different degrees
degrees = [1, 2, 3]
plt.figure(figsize=(12, 10))

for i, degree in enumerate(degrees):
    # Create and fit model without regularization
    model = PolynomialRegression(degree=degree, learning_rate=0.01, 
                               max_iterations=10000, regularization=0.0)
    model.fit(X_train_2d, y_train)
    
    # Make predictions for training and test sets
    y_train_pred = model.predict(X_train_2d)
    y_test_pred = model.predict(X_test_2d)
    
    # Calculate MSE
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    
    # Create smooth curve for visualization
    x_curve = np.linspace(min(x_data), max(x_data), 100).reshape(-1, 1)
    y_curve = model.predict(x_curve)
    
    # Plot
    plt.subplot(2, 2, i+1)
    plt.plot(x_data, y_true, 'g-', label='True function', linewidth=2)
    plt.scatter(X_train, y_train, color='blue', s=30, alpha=0.7, label='Training')
    plt.scatter(X_test, y_test, color='red', s=30, alpha=0.7, label='Test')
    plt.plot(x_curve, y_curve, 'k-', label=f'Degree {degree} model', linewidth=2)
    plt.title(f'Polynomial Degree {degree}\nTrain MSE: {train_mse:.3f}, Test MSE: {test_mse:.3f}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.grid(True)
    
plt.tight_layout()
plt.show()

print("Notice how higher degree polynomials fit the training data better")
print("but perform worse on test data - this is overfitting!")

## 2. L2 Regularization (Ridge Regression)

L2 regularization adds a penalty term to the cost function proportional to the sum of squared weights:

$$J(\mathbf{w}, b) = \frac{1}{2m}\sum_{i=1}^{m}(f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 + \frac{\lambda}{2m}\sum_{j=1}^{n}w_j^2$$

Where:
- $J(\mathbf{w}, b)$ is the regularized cost function
- The first term is the standard mean squared error
- The second term is the regularization penalty
- $\lambda$ is the regularization parameter that controls penalty strength

Let's apply regularization to our highest degree polynomial model and see how it affects performance:

In [ ]:
# Apply different levels of regularization to a high-degree polynomial
high_degree = 3  # High degree to demonstrate overfitting
lambdas = [0, 0.001, 0.1, 10.0]

plt.figure(figsize=(12, 10))

for i, lambda_val in enumerate(lambdas):
    # Create and fit regularized model
    model = PolynomialRegression(degree=high_degree, learning_rate=0.01, 
                               max_iterations=10000, regularization=lambda_val)
    model.fit(X_train_2d, y_train)
    
    # Make predictions
    y_train_pred = model.predict(X_train_2d)
    y_test_pred = model.predict(X_test_2d)
    
    # Calculate MSE
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    
    # Calculate sum of squared weights (excluding bias)
    weight_norm = np.sum(model.weights**2)
    
    # Create smooth curve for visualization
    x_curve = np.linspace(min(x_data), max(x_data), 100).reshape(-1, 1)
    y_curve = model.predict(x_curve)
    
    # Plot
    plt.subplot(2, 2, i+1)
    plt.plot(x_data, y_true, 'g-', label='True function', linewidth=2)
    plt.scatter(X_train, y_train, color='blue', s=30, alpha=0.7, label='Training')
    plt.scatter(X_test, y_test, color='red', s=30, alpha=0.7, label='Test')
    plt.plot(x_curve, y_curve, 'k-', label=f'Degree {high_degree}, λ={lambda_val}', linewidth=2)
    plt.title(f'L2 Regularization (λ={lambda_val})\nTrain MSE: {train_mse:.3f}, Test MSE: {test_mse:.3f}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.ylim(-2.5, 2.5)  # Consistent y-axis for better comparison
    plt.legend()
    plt.grid(True)
    
plt.tight_layout()
plt.show()

## 3. Effect of Regularization on Model Weights

Regularization works by shrinking the model weights toward zero, with higher λ values causing more shrinkage. Let's examine how regularization affects the distribution of weights:

In [ ]:
# Expanded range of regularization values
lambda_values = [0, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
weight_norms = []
train_errors = []
test_errors = []
models = []

for lam in lambda_values:
    # Train model with this regularization strength
    model = PolynomialRegression(degree=high_degree, learning_rate=0.01, 
                               max_iterations=10000, regularization=lam)
    model.fit(X_train_2d, y_train)
    models.append(model)
    
    # Calculate weight norm (L2 norm squared)
    weight_norm = np.sum(model.weights**2)
    weight_norms.append(weight_norm)
    
    # Calculate MSE on training and test sets
    train_pred = model.predict(X_train_2d)
    test_pred = model.predict(X_test_2d)
    train_mse = mean_squared_error(y_train, train_pred)
    test_mse = mean_squared_error(y_test, test_pred)
    train_errors.append(train_mse)
    test_errors.append(test_mse)

# Plot weight magnitudes vs regularization parameter
plt.figure(figsize=(15, 5))

# Plot 1: Weight norm vs regularization strength
plt.subplot(1, 2, 1)
plt.loglog(lambda_values, weight_norms, 'bo-', linewidth=2)
plt.xlabel('Regularization parameter (λ)')
plt.ylabel('Sum of squared weights')
plt.title('Effect of Regularization on Weight Magnitude')
plt.grid(True)

# Plot 2: Training and test errors vs regularization strength
plt.subplot(1, 2, 2)
plt.semilogx(lambda_values, train_errors, 'bo-', linewidth=2, label='Training MSE')
plt.semilogx(lambda_values, test_errors, 'ro-', linewidth=2, label='Test MSE')
plt.xlabel('Regularization parameter (λ)')
plt.ylabel('Mean Squared Error')
plt.title('Error vs Regularization Strength')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

# Find the optimal lambda value (minimizing test error)
best_idx = np.argmin(test_errors)
best_lambda = lambda_values[best_idx]
print(f"Optimal regularization parameter (λ): {best_lambda}")
print(f"Test MSE at optimal λ: {test_errors[best_idx]:.4f}")
print(f"Training MSE at optimal λ: {train_errors[best_idx]:.4f}")

Let's visualize how individual weights change with different regularization strengths:

In [ ]:
# Extract weights from each model
all_weights = np.array([model.weights for model in models])

# Plot regularization path
plt.figure(figsize=(12, 6))
for i in range(all_weights.shape[1]):
    plt.semilogx(lambda_values, all_weights[:, i], '.-', label=f'w{i+1}')
    
plt.axvline(x=best_lambda, color='black', linestyle='--', label='Optimal λ')
plt.xlabel('Regularization parameter (λ)')
plt.ylabel('Weight value')
plt.title('Regularization Path: Weight Values vs λ')
plt.grid(True)

# Only show a subset of weights in the legend to avoid overcrowding
handles, labels = plt.gca().get_legend_handles_labels()
subset = list(range(0, len(handles), max(1, len(handles) // 10)))
subset = subset + [len(handles)-1]  # Add the optimal lambda line
plt.legend([handles[i] for i in subset], [labels[i] for i in subset], loc='best')

plt.tight_layout()
plt.show()

## 4. Model Performance with Optimal Regularization

Now let's compare the model with no regularization, too much regularization, and optimal regularization:

In [ ]:
# Get models with different regularization settings
no_reg_idx = 0  # λ = 0
optimal_reg_idx = best_idx  # Optimal λ
high_reg_idx = len(lambda_values) - 1  # Highest λ

# Extract the models
model_no_reg = models[no_reg_idx]
model_optimal = models[optimal_reg_idx]
model_high_reg = models[high_reg_idx]

# Create smooth curve for visualization
x_curve = np.linspace(min(x_data) - 1, max(x_data) + 1, 200).reshape(-1, 1)
y_no_reg = model_no_reg.predict(x_curve)
y_optimal = model_optimal.predict(x_curve)
y_high_reg = model_high_reg.predict(x_curve)

# Plot comparison
plt.figure(figsize=(12, 6))
plt.plot(x_data, y_true, 'g-', linewidth=3, label='True function')
plt.plot(x_curve, y_no_reg, 'r--', linewidth=2, label=f'No regularization (λ=0), Test MSE: {test_errors[no_reg_idx]:.4f}')
plt.plot(x_curve, y_optimal, 'b-', linewidth=2, label=f'Optimal regularization (λ={best_lambda}), Test MSE: {test_errors[optimal_reg_idx]:.4f}')
plt.plot(x_curve, y_high_reg, 'k--', linewidth=2, label=f'High regularization (λ={lambda_values[high_reg_idx]}), Test MSE: {test_errors[high_reg_idx]:.4f}')
plt.scatter(X_train, y_train, color='blue', s=50, alpha=0.5, label='Training data')
plt.scatter(X_test, y_test, color='red', s=50, alpha=0.5, label='Test data')

plt.title(f'Comparison of Models with Different Regularization (Degree {high_degree})')
plt.xlabel('x')
plt.ylabel('y')
plt.ylim(-3, 3)  # Set consistent y-axis bounds
plt.legend()
plt.grid(True)
plt.show()

## 5. Combining Model Complexity and Regularization

Now let's explore how model complexity (polynomial degree) and regularization interact. We'll find the optimal combination of both hyperparameters:

In [ ]:
# Grid search over polynomials and regularization
degrees_grid = [1, 2, 3]
lambdas_grid = [0, 0.001, 0.01, 0.1, 1.0, 10.0]

# Initialize results grid
results = np.zeros((len(degrees_grid), len(lambdas_grid)))

# Train all combinations
for i, degree in enumerate(degrees_grid):
    for j, lambda_val in enumerate(lambdas_grid):
        # Create and train model
        model = PolynomialRegression(degree=degree, learning_rate=0.01, 
                                   max_iterations=10000, regularization=lambda_val)
        model.fit(X_train_2d, y_train)
        
        # Calculate test error
        y_test_pred = model.predict(X_test_2d)
        test_mse = mean_squared_error(y_test, y_test_pred)
        
        # Store result
        results[i, j] = test_mse

# Find the best combination
min_idx = np.unravel_index(np.argmin(results), results.shape)
best_degree = degrees_grid[min_idx[0]]
best_lambda = lambdas_grid[min_idx[1]]

# Plot results as a heatmap
plt.figure(figsize=(10, 8))
plt.imshow(results, cmap='viridis', aspect='auto', origin='lower')
plt.colorbar(label='Test MSE')
plt.title('Test MSE for Different Combinations of Degree and Regularization')
plt.xlabel('Regularization Parameter (λ)')
plt.ylabel('Polynomial Degree')
plt.xticks(np.arange(len(lambdas_grid)), lambdas_grid)
plt.yticks(np.arange(len(degrees_grid)), degrees_grid)

# Mark the best combination
plt.plot(min_idx[1], min_idx[0], 'ro', markersize=12)

# Add text annotations with error values
for i in range(len(degrees_grid)):
    for j in range(len(lambdas_grid)):
        plt.text(j, i, f'{results[i, j]:.2f}', ha='center', va='center', 
                color='white' if results[i, j] > np.median(results) else 'black')

plt.tight_layout()
plt.show()

print(f"Best combination: Polynomial degree = {best_degree}, Regularization λ = {best_lambda}")
print(f"Test MSE with optimal settings: {results[min_idx]:.4f}")

## 6. Visualizing the Optimal Model

Let's train the model with the optimal degree and regularization parameters and visualize its performance:

In [ ]:
# Train the optimal model
optimal_model = PolynomialRegression(degree=best_degree, learning_rate=0.01, 
                                   max_iterations=10000, regularization=best_lambda)
optimal_model.fit(X_train_2d, y_train)

# Calculate errors
y_train_pred = optimal_model.predict(X_train_2d)
y_test_pred = optimal_model.predict(X_test_2d)
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

# Create smooth curve for visualization
x_curve = np.linspace(min(x_data) - 1, max(x_data) + 1, 200).reshape(-1, 1)
y_curve = optimal_model.predict(x_curve)

# Plot optimal model
plt.figure(figsize=(10, 6))
plt.plot(x_data, y_true, 'g-', linewidth=3, label='True function')
plt.scatter(X_train, y_train, color='blue', s=50, alpha=0.7, label='Training data')
plt.scatter(X_test, y_test, color='red', s=50, alpha=0.7, label='Test data')
plt.plot(x_curve, y_curve, 'k-', linewidth=2, label='Optimal model')

plt.title(f'Optimal Model: Degree {best_degree}, λ={best_lambda}\nTrain MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()

## 7. Conclusion: Key Regularization Insights

### What We've Learned

1. **The Overfitting Problem**
   - Without regularization, complex models (high-degree polynomials) tend to overfit training data
   - Overfitted models have low training error but high test error
   - Overfitting captures noise rather than the underlying pattern

2. **L2 Regularization (Ridge)**
   - Adds a penalty term proportional to the sum of squared weights
   - Controls model complexity without reducing the number of features
   - Shrinks weights toward zero, but typically doesn't eliminate them completely
   - Controlled by the regularization parameter λ

3. **Effect of Regularization Strength**
   - λ = 0: No regularization, potential overfitting
   - Small λ: Slight regularization, still flexible
   - Optimal λ: Best balance between fitting data and constraining weights
   - Large λ: Heavy regularization, can lead to underfitting

4. **Model Complexity vs. Regularization**
   - Higher complexity models (higher polynomial degrees) need stronger regularization
   - Lower complexity models need less regularization
   - The optimal combination provides the best generalization performance

5. **Practical Guidelines**
   - Use validation/test data to find optimal regularization strength
   - Consider a grid search over both model complexity and regularization parameters
   - Monitor both training and test error to detect overfitting
   - Smaller weights generally lead to better generalization

Regularization is a powerful technique that allows us to use complex models while preventing overfitting, resulting in models that generalize better to new, unseen data.